In [5]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# Set random seed for reproducibility
manualSeed = 500 #999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  500


In [ ]:
dataroot = "/celeba"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 256 #128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 2

# Learning rate for optimizers
lr = 0.000002 #0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

pretrained_generator_model_path = '/content/drive/MyDrive/CV_Proj/DCGAN_generator.pth'
pretrained_discriminator_model_path = '/content/drive/My Drive/DCGAN_discriminator_model_6.pth'

In [6]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [8]:
import torch.nn as nn 
from cnn_finetune import make_model

def make_classifier(in_features, num_classes):
  return nn.Sequential(nn.Linear(in_features,num_classes), nn.Sigmoid())



disc_3 = make_model('xception', num_classes=1, pretrained=False, input_size=(64,64), classifier_factory = make_classifier)


In [9]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu

        self.model = disc_3.to(device)
        
    def forward(self, input):
        return self.model(input)

In [12]:
batch_size = 32
momentum = 0.9
# lr = 0.001
epochs = 10
log_interval = 10

count1 = 0
count2 = 0

X = []
y = []

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 64

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

ngpu = 0
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

model_G = Generator(ngpu)#.to(device)
model_G.load_state_dict(torch.load('/content/drive/MyDrive/CV_Proj/DCGAN_generator.pth', map_location=torch.device('cpu'))) #.to(device)

for i in range(50):
  count1 += 64 
  fixed_noise = torch.randn(64, nz, 1, 1, device=device)
  with torch.no_grad():
    fake = model_G(fixed_noise).detach().cpu()
  X.append(fake)
  y.append(torch.zeros(len(fake)))

X =torch.cat(X, 0)
y = torch.cat(y,0)
print(X.shape, y.shape)

cpu
torch.Size([3200, 3, 64, 64]) torch.Size([3200])


In [13]:
dataroot = "/celeba"
import glob
count2 = 0
import numpy as np 
from PIL import Image

transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
for i in glob.glob(dataroot + '/img_align_celeba/img_align_celeba/*'):
  count2+=1 
  if (count2<=3200):
    im = Image.open(i)
    im = transform(im)
    im = im.unsqueeze(0)
    
    X = torch.cat((X,im ), 0)
    y = torch.cat((y, torch.tensor(1).unsqueeze(0)), 0)
  if(count2>=3200):
    break 

print(count1, count2)

3200 3200


In [14]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import cv2 
import glob


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

X_train, X_test, y_train, y_test = torch.tensor(X_train), torch.tensor(X_test), torch.tensor(y_train), torch.tensor(y_test) 


X_train.shape


my_dataset_train = TensorDataset(X_train, y_train) # create your datset
train_loader = DataLoader(my_dataset_train, batch_size=32, shuffle=True)

my_dataset_test = TensorDataset(X_test, y_test) # create your datset
test_loader = DataLoader(my_dataset_test, batch_size=32, shuffle=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


In [15]:
model = Discriminator(ngpu=1).to(device)

In [17]:
lr = 0.0005
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

batch_size = 64
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_func = nn.BCELoss()

def train(epoch):
    model.train()
    correct =0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).float(), target.to(device).float()
        
        optimizer.zero_grad()
        output = model(data).view(-1).to(device)
        loss = loss_func(output, target.to(device).float())
        loss.backward()
        optimizer.step()
        pred = torch.zeros(output.data.shape).to(device)
        pred[output.data >=0.5] = 1

        correct += pred.eq(target.data.view_as(pred)).cpu().sum().to(device)
    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy{:.3f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item(), 100. * correct / len(train_loader.dataset) ))

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device).float()
        target = target.to(device).long()
        output = model(data).view(-1)
        validation_loss = loss_func(output, target.float())
        pred = torch.zeros(output.data.shape).to(device)
        pred[output.data >=0.5] = 1
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(test_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, epochs + 1):
    train(epoch)
    validation()
    model_file = 'DCGAN_discriminator_model_' + str(epoch) + '.pth'
    torch.save(model, model_file)
    print('\nSaved model to ' + model_file + '.')

Train Epoch: 1 [5408/5440 (99%)]	Loss: 0.202747	Accuracy88.346

Validation set: Average loss: 0.0005, Accuracy: 782/960 (81%)


Saved model to DCGAN_discriminator_model_1.pth.
Train Epoch: 2 [5408/5440 (99%)]	Loss: 0.019160	Accuracy98.290

Validation set: Average loss: 0.0000, Accuracy: 950/960 (99%)


Saved model to DCGAN_discriminator_model_2.pth.
Train Epoch: 3 [5408/5440 (99%)]	Loss: 0.000436	Accuracy98.971

Validation set: Average loss: 0.0000, Accuracy: 949/960 (99%)


Saved model to DCGAN_discriminator_model_3.pth.
Train Epoch: 4 [5408/5440 (99%)]	Loss: 0.149118	Accuracy99.375

Validation set: Average loss: 0.0000, Accuracy: 908/960 (95%)


Saved model to DCGAN_discriminator_model_4.pth.
Train Epoch: 5 [5408/5440 (99%)]	Loss: 0.001623	Accuracy99.081

Validation set: Average loss: 0.0000, Accuracy: 953/960 (99%)


Saved model to DCGAN_discriminator_model_5.pth.
Train Epoch: 6 [5408/5440 (99%)]	Loss: 0.000150	Accuracy99.890

Validation set: Average loss: 0.0000, Accuracy: 957/960 